In [1]:
import ee
import geopandas as gpd
from raster_classes.EERasterBuilder import EERasterBuilder
from raster_classes.S2CompositeRaster import S2CompositeRaster
from raster_classes.S2index import S2index
from raster_classes.FeatureExtraction import FeatureExtraction
import pandas as pd

import time
import logging
import os

In [2]:
def init_log(fn_name):
    # Create logger
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    # Create console handler and set level to INFO
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)

    # Create file handler and set level to INFO
    fh = logging.FileHandler(fn_name)
    fh.setLevel(logging.INFO)

    # Create formatter
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    # Add formatter to handlers
    ch.setFormatter(formatter)
    fh.setFormatter(formatter)

    # Add handlers to logger
    logger.addHandler(ch)
    logger.addHandler(fh)

    logger.propagate = False
    return logger

In [3]:
def create_directory(directory_name):
    current_dir = os.getcwd()  # get the current working directory
    new_dir_path = os.path.join(current_dir, directory_name)  # create the full path for the new directory
    if not os.path.exists(new_dir_path):
        os.makedirs(new_dir_path)

In [4]:
def get_date_list(start_date, end_date, time_window):
    """
    To get list of start and dates for the given date ranges
    :param start_date: '2021-01-01'
    :param end_date: '2022-01-01'
    :param time_window: 30
    :return: start_date_li, end_date_li
    """
    freq = str(time_window)+'D'
    df = pd.DataFrame({'start_date': pd.date_range(start=start_date, end=end_date, freq=freq)})
    df['end_date'] = df['start_date'] + pd.Timedelta(days=time_window)
    df = df[(df['end_date'] <= end_date)]
    start_li = list(df.start_date.apply(lambda x: x.strftime('%Y-%m-%d')).astype(str))
    end_li = list(df.end_date.apply(lambda x: x.strftime('%Y-%m-%d')).astype(str))
    return start_li, end_li


In [5]:
# ee.Authenticate()

In [6]:
import ee

ee.Authenticate()  # Initiate authentication

# Rest of your Earth Engine code using ee


True

In [ ]:
year_li = [2018]
for i in year_li:
    if __name__ == '__main__':
        logger = init_log(str(i) + '_log.txt')
        create_directory(str(i))
        start_date = str(i)+'-09-01'
        end_date = str(i+1)+'-09-01'
        ee.Initialize()
        _start_time = time.time()
        time_window = 20
        asset_path = 'users/janardanroy/MNCFC_23/IND_RABI_17_18_19_UPDATED'
        start_date_list, end_date_list = get_date_list(start_date, end_date, time_window)
        bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

        for start, end in zip(start_date_list, end_date_list):
            s2 = S2CompositeRaster(start=start, end=end,
                              bands=bands,
                              cloud_filter=100,
                              cloudThresh=20)

            s2_indices = S2index(s2rc=s2, index=['NDVI', 'NDRE', 'LSWI', 'GCVI', 'SAVI', 'BSI'])

        rcls_list = EERasterBuilder().instance_list
        logger.info('No. of raster classes: %s', len(rcls_list))

        fc = ee.FeatureCollection(asset_path)
        fc_yr = fc.filter(ee.Filter.eq('sowing_yea', i))
        reg_li = fc_yr.aggregate_array('state').distinct().getInfo()
        # reg_li = ['andhrapradesh', 'bihar', 'gujarat',
        #           'haryana', 'jharkhand', 'karnataka', 'kerala',
        #           'madhyapradesh', 'maharashtra', 'punjab', 'rajasthan',
        #           'tamilnadu', 'telangana', 'uttarpradesh', 'westbengal']

        logger.info('State list: %s', reg_li)

        for reg in reg_li:
            logger.info('Downloading for %s', reg)
            reg_fc = fc_yr.filter(ee.Filter.eq('state', reg)).sort('district')
            collection_size = reg_fc.size().getInfo()
            logger.info('Total no.of plots in {}: {}' .format(reg, collection_size))
            chunk_size = 500
            for chunk in range(0, collection_size, chunk_size):
                output_csv_file = reg + str('_') + str(i) + str('_') + str(chunk + chunk_size) + '.csv'
                logger.info('output file name: %s', output_csv_file)
                loop_start_time = time.time()
                subset_col = ee.FeatureCollection(reg_fc.toList(chunk_size, chunk))
                fe = FeatureExtraction(fc=subset_col, rc_list=rcls_list, file_name=output_csv_file[0:-4], logger=logger,sampleRegions=reg)
                total_df = fe.execute()
                total_df.to_csv(str(i)+'/'+output_csv_file)
                elapsed_time = time.time() - loop_start_time
                logger.info('Time taken to process {} plots is {} min' .format(chunk+chunk_size, elapsed_time / 60))
            elapsed_time = time.time() - _start_time
            logger.info('Total Processing time (hrs) to complete {}: {}' .format(reg, elapsed_time / 3600))

2024-04-30 13:30:28,324 - INFO - No. of raster classes: 36
2024-04-30 13:30:28,658 - INFO - State list: ['bihar', 'kerala', 'orissa', 'punjab', 'gujarat', 'haryana', 'jharkhand', 'karnataka', 'meghalaya', 'rajasthan', 'tamilnadu', 'telangana', 'westbengal', 'chattisgarh', 'maharashtra', 'uttarpradesh', 'andhrapradesh', 'madhyapradesh', 'arunachalpradesh']
2024-04-30 13:30:28,659 - INFO - Downloading for bihar
2024-04-30 13:30:29,179 - INFO - Total no.of plots in bihar: 11291
2024-04-30 13:30:29,180 - INFO - output file name: bihar_2018_500.csv
2024-04-30 13:31:13,034 - INFO - Processing time for bihar_2018_500_t0 is 0.7309050559997559 min
2024-04-30 13:33:11,749 - INFO - Processing time for bihar_2018_500_t1 is 1.9785545706748962 min
2024-04-30 13:35:11,424 - INFO - Processing time for bihar_2018_500_t2 is 1.9945746024449666 min
2024-04-30 13:36:50,870 - INFO - Processing time for bihar_2018_500_t3 is 1.6574248592058818 min
2024-04-30 13:37:48,819 - INFO - Processing time for bihar_201